# 0. Setup Paths

In [5]:
import os
import wget
import object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [6]:
# indicate custom model & desired checkpoint from training
CUSTOM_MODEL = 'my_centernet_hg104_1024_7_TestNikon'
PRETRAINED_CUSTOM_MODEL = 'my_centernet_hg104_1024_7'
CUSTOM_CHECKPOINT = 'ckpt-21'
LABEL_MAP_NAME = 'label_map.pbtxt'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'

In [7]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations',CUSTOM_MODEL),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'IMAGE_PATH_TRAIN': os.path.join('Tensorflow', 'workspace','images','train'),
    'IMAGE_PATH_TEST': os.path.join('Tensorflow', 'workspace','images','test'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'PRETRAINED_CUSTOM_MODEL_PATH': os.path.join('Tensorflow', 'workspace','models',PRETRAINED_CUSTOM_MODEL),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [8]:
files = {
    'PRETRAINED_PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', PRETRAINED_CUSTOM_MODEL, 'pipeline.config'),
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [9]:
# creating folders
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [10]:
#VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
#!python {VERIFICATION_SCRIPT}

In [11]:
#!pip list

In [12]:
# CHECKPOINT PATH: copy pipeline.config (pretrained model) into new config folder
if os.name =='posix':
    !cp {files['PRETRAINED_PIPELINE_CONFIG']} {files['PIPELINE_CONFIG']}
if os.name == 'nt':
    !copy {files['PRETRAINED_PIPELINE_CONFIG']} {files['PIPELINE_CONFIG']}

In [13]:
labels = [{'name':'Bubble', 'id':1}]

# write into label map file
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [14]:
# create TFRecord file
# --> train and test record file within annotations path
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 


Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_7_TestNikon/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_7_TestNikon/test.record


In [15]:
# Load pipeline config (copy of pretrained custom model config file)
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)

In [16]:
# Load pipeline config (new/modified config file)
#configs_test = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

# 2. Create Label Map

In [17]:
# modify pipeline.config for personal use case (num classes, set paths etc.)
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_CUSTOM_MODEL_PATH'],CUSTOM_CHECKPOINT)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [18]:
print([os.path.join(paths['ANNOTATION_PATH'], 'test.record')])

['Tensorflow/workspace/annotations/my_centernet_hg104_1024_7_TestNikon/test.record']


In [19]:
# save new pipeline config (to model folder)
config_text = text_format.MessageToString(pipeline_config)
# dfile: accessing the file ('wb'= writing to file in binary mode)                                                                                                                                                                                                     
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

In [20]:
# Load pipeline config (new/modified config file)
configs_new = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [21]:
configs_new

{'model': center_net {
   num_classes: 1
   feature_extractor {
     type: "hourglass_104"
     channel_means: 104.01361846923828
     channel_means: 114.03422546386719
     channel_means: 119.91659545898438
     channel_stds: 73.60276794433594
     channel_stds: 69.89082336425781
     channel_stds: 70.91507720947266
     bgr_ordering: true
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 1024
       max_dimension: 1024
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 100
   }
 },
 'train_config': batch_si

# 6. Train the model
Before Training, my_model directory must be empty!

In [22]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [23]:
# test if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# check GPU use
# watch -n0.1 nvidia-smi

Num GPUs Available:  1


In [24]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=100".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [25]:
# to see training status, copy command to prompt and run from there
# (from tfodCourse directory and with activated environment)
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_7_TestNikon --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_7_TestNikon/pipeline.config --num_train_steps=100


# 7. Evaluate the Model
Calculate performance metrics

In [45]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [46]:
# command can be run from prompt to see progress
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_7_v2 --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_7_v2/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_7_v2
